# Telco Project
***
### Project Description
Accurately predict customer churn using machine learning classification
algorithms

### Table of Contents:
1. [Planning](#Planning)
2. [Acquisition](#Acquisition)
3. [Preparation](#Preparation)
4. [Exploration](#Exploration)
5. [Modeling](#Modeling)
6. [Delivery](#Delivery)

## Planning
---
 - [ ] Goal(s)
     - [ ] Find drivers of customer churn
     - [ ] Accurately predict customer churn at Telco.
 - [ ] Measure(s) of success
     - [ ] Hypothesis testing
     - [ ] Baseline accuracy
     - [ ] 3 classification models
         - [ ] Model performance: train, validate, test
         - [ ] Hyperparameter tuning
 - [ ] Plan to achieve 1 & 2
 - [ ] Develop hypotheses
     - [ ] Brainstorm questions
 
Brainstorming questions to form hypothesis:
Do customer's of a certain demographic churn more than the rest?
Does service package influence churn?
Does having add-on services influence whether a customer churns?
Does the payment method influence whether customers churn?


## Acquisition
--------------
- [ ] Instructions to acquire data
- [x] Upload `.csv` file to repository - file named `telecom_data.csv`
- [ ] acquire.py file
    1. [x] Write functions to acquire telco dataset
    2. [ ] Write docstring for each function

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from acquire import get_telco_data
from prepare import telco_data_prep, preprocessed_data, data_target_splitter
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [2]:
# df_telco.nunique()

In [3]:
# object_columns = df_telco.nunique()[df_telco.nunique() <= 4]

In [4]:
# object_columns = object_columns.index.to_list()

In [5]:
# for column in object_columns:
#     print(df_telco[column].value_counts().sort_index())
#     print('')

In [6]:
# sns.distplot(df_telco.tenure);

In [7]:
# sns.distplot(df_telco.monthly_charges);

## Preparation and Processing
---
- [ ] Document Process

## Exploration
---
- [ ] Statistical Analysis
    - [ ] Restate hypothesis here
    - [ ] Test hypotheses
    - [ ] Plot distributions
- [ ] Create visuals
- [ ] Present and summarize key findings

Hypotheses


In [39]:
train, validate, test = preprocessed_data()

In [42]:
df_numeric_attributes = train.select_dtypes(exclude='uint8')
df_categorical_attributes = train.select_dtypes(exclude=['float64', 'int64'])

numeric_columns = df_numeric_attributes.columns.to_list().remove('churn')
categorical_columns = df_categorical_attributes.columns.to_list()

In [ ]:
sns.pairplot(train,
             x_vars = numeric_columns,
             y_vars = numeric_columns,
             hue = 'churn');

In [13]:
sns.boxenpot(train.)

Index(['gender_Male', 'partner_Yes', 'dependents_Yes', 'phone_service_Yes',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'online_security_No internet service', 'online_security_Yes',
       'online_backup_No internet service', 'online_backup_Yes',
       'device_protection_No internet service', 'device_protection_Yes',
       'tech_support_No internet service', 'tech_support_Yes',
       'streaming_tv_No internet service', 'streaming_tv_Yes',
       'streaming_movies_No internet service', 'streaming_movies_Yes',
       'paperless_billing_Yes', 'internet_service_type_Fiber optic',
       'internet_service_type_None', 'contract_type_One year',
       'contract_type_Two year', 'payment_type_Credit card (automatic)',
       'payment_type_Electronic check', 'payment_type_Mailed check'],
      dtype='object')

## Modeling
---
- [ ] sklearn.domymathhomework.classification_models
    - [ ] Create 3 classification models

## Delivery
---

- [ ] Summarize/Recap key findings
    - [ ] Drivers

[Return to the top](#Telco-Project)